In [11]:
pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 19.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


OSError: libgomp.so.1: cannot open shared object file: No such file or directory

In [3]:
# Build a .py script that takes a snapshot date, trains a model and outputs artefact into storage.

## set up pyspark session

In [4]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 07:13:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/12 07:13:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## set up config

In [5]:
# set up config
model_train_date_str = "2024-09-01"
train_test_period_months = 12
oot_period_months = 2
train_test_ratio = 0.8

config = {}
config["model_train_date_str"] = model_train_date_str
config["train_test_period_months"] = train_test_period_months
config["oot_period_months"] =  oot_period_months
config["model_train_date"] =  datetime.strptime(model_train_date_str, "%Y-%m-%d")
config["oot_end_date"] =  config['model_train_date'] - timedelta(days = 1)
config["oot_start_date"] =  config['model_train_date'] - relativedelta(months = oot_period_months)
config["train_test_end_date"] =  config["oot_start_date"] - timedelta(days = 1)
config["train_test_start_date"] =  config["oot_start_date"] - relativedelta(months = train_test_period_months)
config["train_test_ratio"] = train_test_ratio 


pprint.pprint(config)

{'model_train_date': datetime.datetime(2024, 9, 1, 0, 0),
 'model_train_date_str': '2024-09-01',
 'oot_end_date': datetime.datetime(2024, 8, 31, 0, 0),
 'oot_period_months': 2,
 'oot_start_date': datetime.datetime(2024, 7, 1, 0, 0),
 'train_test_end_date': datetime.datetime(2024, 6, 30, 0, 0),
 'train_test_period_months': 12,
 'train_test_ratio': 0.8,
 'train_test_start_date': datetime.datetime(2023, 7, 1, 0, 0)}


## get label store

In [6]:
# connect to label store
folder_path = "datamart/gold/label_store/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
label_store_sdf = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",label_store_sdf.count())

label_store_sdf.show()

row_count: 8974
+--------------------+-----------+-----+----------+-------------+
|             loan_id|Customer_ID|label| label_def|snapshot_date|
+--------------------+-----------+-----+----------+-------------+
|CUS_0x1037_2023_0...| CUS_0x1037|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1069_2023_0...| CUS_0x1069|    0|30dpd_6mob|   2023-07-01|
|CUS_0x114a_2023_0...| CUS_0x114a|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1184_2023_0...| CUS_0x1184|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1297_2023_0...| CUS_0x1297|    1|30dpd_6mob|   2023-07-01|
|CUS_0x12fb_2023_0...| CUS_0x12fb|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1325_2023_0...| CUS_0x1325|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1341_2023_0...| CUS_0x1341|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1375_2023_0...| CUS_0x1375|    1|30dpd_6mob|   2023-07-01|
|CUS_0x13a8_2023_0...| CUS_0x13a8|    0|30dpd_6mob|   2023-07-01|
|CUS_0x13ef_2023_0...| CUS_0x13ef|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1440_2023_0...| CUS_0x1440|    0|30dpd_6mob|   2023-0

In [6]:
# extract label store
labels_sdf = label_store_sdf.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))

print("extracted labels_sdf", labels_sdf.count(), config["train_test_start_date"], config["oot_end_date"])

extracted labels_sdf 6961 2023-07-01 00:00:00 2024-08-31 00:00:00


## get features

In [7]:
# connect to feature store
folder_path_1 = "datamart/gold/feature_store/eng/"
folder_path_2 = "datamart/gold/feature_store/cust_fin_risk/"
files_list_1 = [folder_path_1+os.path.basename(f) for f in glob.glob(os.path.join(folder_path_1, '*'))]
files_list_2 = [folder_path_2+os.path.basename(f) for f in glob.glob(os.path.join(folder_path_2, '*'))]
feature_store_sdf_1 = spark.read.option("header", "true").parquet(*files_list_1)
feature_store_sdf_2 = spark.read.option("header", "true").parquet(*files_list_2)
print("row_count:",feature_store_sdf_1.count())
print("row_count:",feature_store_sdf_2.count())

print("======Feature Table 1======")
feature_store_sdf_1.show()
print("======Feature Table 2======")
feature_store_sdf_2.show()

row_count: 206402
row_count: 11974
======Feature Table 1======
+-----------+-------------+--------+--------+--------+--------+--------+--------+
|Customer_ID|snapshot_date|click_1m|click_2m|click_3m|click_4m|click_5m|click_6m|
+-----------+-------------+--------+--------+--------+--------+--------+--------+
| CUS_0xc65a|   2024-03-01|     239|     236|      44|     309|       0|      35|
| CUS_0x5e1f|   2024-03-01|     103|      26|     182|     133|      27|      43|
| CUS_0x78d3|   2024-03-01|      69|     124|     140|       0|       0|     149|
| CUS_0x1844|   2024-03-01|     142|     232|      25|     145|       0|      97|
| CUS_0x7f07|   2024-03-01|      53|       0|     256|     158|     160|     141|
| CUS_0xbd2c|   2024-03-01|      93|      70|     318|      49|     329|     155|
| CUS_0x9d41|   2024-03-01|      69|       0|      47|      59|      39|      84|
| CUS_0x834f|   2024-03-01|     240|      49|      16|      24|     171|      52|
| CUS_0x4b6b|   2024-03-01|       8

In [9]:
# extract feature store
features_sdf_1 = feature_store_sdf_1.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))
features_sdf_2 = feature_store_sdf_2.filter((col("snapshot_date") >= config["train_test_start_date"]) & (col("snapshot_date") <= config["oot_end_date"]))

print("extracted features_sdf_1", features_sdf_1.count(), config["train_test_start_date"], config["oot_end_date"])
print("extracted features_sdf_2", features_sdf_2.count(), config["train_test_start_date"], config["oot_end_date"])

extracted features_sdf_1 125636 2023-07-01 00:00:00 2024-08-31 00:00:00
extracted features_sdf_2 6937 2023-07-01 00:00:00 2024-08-31 00:00:00


## prepare data for modeling

In [9]:
# prepare data for modeling
data_pdf = labels_sdf.join(features_sdf, on=["Customer_ID", "snapshot_date"], how="left").toPandas()
data_pdf

,Customer_ID,snapshot_date,loan_id,label,label_def,fe_1,fe_2,fe_3,fe_4,fe_5,...,fe_11,fe_12,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20
0,CUS_0x1015,2024-02-01,CUS_0x1015_2023_08_01,0,30dpd_6mob,-22,128,252,-46,71,...,153,208,-34,9,112,198,29,113,37,-61
1,CUS_0x10eb,2023-09-01,CUS_0x10eb_2023_03_01,0,30dpd_6mob,-26,83,100,16,-23,...,276,124,166,175,-164,277,-18,239,45,-37
2,CUS_0x10ff,2024-04-01,CUS_0x10ff_2023_10_01,0,30dpd_6mob,116,22,62,79,261,...,67,257,225,132,175,-45,255,-106,49,53
3,CUS_0x112f,2024-04-01,CUS_0x112f_2023_10_01,1,30dpd_6mob,222,175,140,116,-63,...,82,196,230,-24,-87,-33,167,206,91,116
4,CUS_0x117d,2024-03-01,CUS_0x117d_2023_09_01,0,30dpd_6mob,0,90,-209,-30,253,...,60,35,245,348,81,99,-123,109,270,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6956,CUS_0xfa4,2024-08-01,CUS_0xfa4_2024_02_01,1,30dpd_6mob,148,192,122,120,81,...,116,318,132,125,58,170,164,-36,195,131
6957,CUS_0xfaa,2024-01-01,CUS_0xfaa_2023_07_01,0,30dpd_6mob,42,19,170,58,118,...,232,156,39,102,247,221,210,142,138,2
6958,CUS_0xfb4,2024-06-01,CUS_0xfb4_2023_12_01,0,30dpd_6mob,182,148,-36,151,5,...,71,254,107,155,53,120,103,246,99,-31
6959,CUS_0xfc9,2023-07-01,CUS_0xfc9_2023_01_01,1,30dpd_6mob,258,-13,189,58,147,...,-40,175,47,69,-85,176,13,-1,96,-14


In [10]:
# split data into train - test - oot
oot_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["oot_start_date"].date()) & (data_pdf['snapshot_date'] <= config["oot_end_date"].date())]
train_test_pdf = data_pdf[(data_pdf['snapshot_date'] >= config["train_test_start_date"].date()) & (data_pdf['snapshot_date'] <= config["train_test_end_date"].date())]

feature_cols = [fe_col for fe_col in data_pdf.columns if fe_col.startswith('fe_')]

X_oot = oot_pdf[feature_cols]
y_oot = oot_pdf["label"]
X_train, X_test, y_train, y_test = train_test_split(
    train_test_pdf[feature_cols], train_test_pdf["label"], 
    test_size= 1 - config["train_test_ratio"],
    random_state=88,     # Ensures reproducibility
    shuffle=True,        # Shuffle the data before splitting
    stratify=train_test_pdf["label"]           # Stratify based on the label column
)


print('X_train', X_train.shape[0])
print('X_test', X_test.shape[0])
print('X_oot', X_oot.shape[0])
print('y_train', y_train.shape[0], round(y_train.mean(),2))
print('y_test', y_test.shape[0], round(y_test.mean(),2))
print('y_oot', y_oot.shape[0], round(y_oot.mean(),2))

X_train

X_train 4766
X_test 1192
X_oot 1003
y_train 4766 0.28
y_test 1192 0.28
y_oot 1003 0.29


,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,fe_11,fe_12,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20
168,88,111,143,63,153,62,235,126,200,-5,162,156,79,11,-2,175,86,-46,266,180
4189,271,115,247,74,148,205,116,22,-50,159,40,71,49,4,144,-43,103,-28,161,104
3405,269,103,207,84,359,120,87,89,94,134,308,145,-34,80,-5,184,191,-138,240,-52
507,224,-33,-95,-71,127,131,153,111,231,-63,108,124,76,163,85,43,188,27,101,182
4274,127,98,164,-125,120,11,-80,220,107,140,-73,46,172,220,95,109,-70,16,93,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,27,-74,26,228,317,80,133,51,227,204,61,190,-103,141,35,121,-15,222,-58,52
4322,117,11,114,231,253,49,33,236,-46,11,152,-208,239,118,61,150,176,147,96,48
2114,176,163,230,39,124,162,-259,164,91,-48,48,5,124,151,189,194,199,91,204,309
6291,-12,71,154,156,-53,161,134,-75,153,233,203,164,208,-38,146,147,2,206,247,80


## preprocess data

In [11]:
# set up standard scalar preprocessing
scaler = StandardScaler()

transformer_stdscaler = scaler.fit(X_train) # Q which should we use? train? test? oot? all?

# transform data
X_train_processed = transformer_stdscaler.transform(X_train)
X_test_processed = transformer_stdscaler.transform(X_test)
X_oot_processed = transformer_stdscaler.transform(X_oot)

print('X_train_processed', X_train_processed.shape[0])
print('X_test_processed', X_test_processed.shape[0])
print('X_oot_processed', X_oot_processed.shape[0])

pd.DataFrame(X_train_processed)

X_train_processed 4766
X_test_processed 1192
X_oot_processed 1003


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.131264,0.098858,0.371632,-0.407039,0.433536,-0.397983,1.276021,0.150990,0.871354,-1.217275,0.603611,0.578890,-0.218767,-0.896619,-1.021965,0.722443,-0.151516,-1.457934,1.606529,0.787418
1,1.692313,0.138639,1.413284,-0.297619,0.384094,1.012099,0.096916,-0.899325,-1.669252,0.416730,-0.599955,-0.282965,-0.512331,-0.966447,0.432676,-1.450095,0.014703,-1.282612,0.575401,0.056203
2,1.672383,0.019296,1.012649,-0.198145,2.470546,0.173938,-0.190429,-0.222680,-0.205863,0.167643,2.043944,0.467356,-1.324525,-0.208309,-1.051855,0.812135,0.875134,-2.354026,1.351202,-1.444711
3,1.223962,-1.333265,-2.012148,-1.739982,0.176437,0.282406,0.463529,-0.000498,1.186389,-1.795155,0.070885,0.254427,-0.248123,0.619657,-0.155159,-0.593039,0.845801,-0.746906,-0.013816,0.806661
4,0.257367,-0.030431,0.581966,-2.277138,0.107218,-0.900879,-1.845139,1.100313,-0.073751,0.227424,-1.714734,-0.536452,0.691282,1.188261,-0.055526,0.064702,-1.676825,-0.854047,-0.092378,1.201132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4761,-0.739124,-1.741022,-0.800226,1.234271,2.055233,-0.220490,0.265360,-0.606449,1.145740,0.865085,-0.392784,0.923633,-1.999722,0.400196,-0.653323,0.184292,-1.139056,1.152418,-1.575238,-0.444101
4762,0.157718,-0.895672,0.081172,1.264113,1.422375,-0.526172,-0.725485,1.261899,-1.628602,-1.057860,0.504958,-3.111879,1.346908,0.170760,-0.394278,0.473299,0.728470,0.421909,-0.062917,-0.482586
4763,0.745647,0.616014,1.243014,-0.645775,0.146772,0.588088,-3.618751,0.534759,-0.236350,-1.645703,-0.521033,-0.952170,0.221579,0.499951,0.881024,0.911793,0.953355,-0.123538,0.997672,2.028559
4764,-1.127755,-0.298954,0.481807,0.518063,-1.603475,0.578227,0.275268,-1.878945,0.393720,1.154024,1.008088,0.660006,1.043559,-1.385419,0.452602,0.443402,-0.972837,0.996576,1.419944,-0.174707


## train model

In [12]:
# Define the XGBoost classifier
xgb_clf = xgb.XGBClassifier(eval_metric='logloss', random_state=88)

# Define the hyperparameter space to search
param_dist = {
    'n_estimators': [25, 50],
    'max_depth': [2, 3],  # lower max_depth to simplify the model
    'learning_rate': [0.01, 0.1],
    'subsample': [0.6, 0.8],
    'colsample_bytree': [0.6, 0.8],
    'gamma': [0, 0.1],
    'min_child_weight': [1, 3, 5],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

# Create a scorer based on AUC score
auc_scorer = make_scorer(roc_auc_score)

# Set up the random search with cross-validation
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_dist,
    scoring=auc_scorer,
    n_iter=100,  # Number of iterations for random search
    cv=3,       # Number of folds in cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform the random search
random_search.fit(X_train_processed, y_train)

# Output the best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print("Best AUC score: ", random_search.best_score_)

# Evaluate the model on the train set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_train_processed)[:, 1]
train_auc_score = roc_auc_score(y_train, y_pred_proba)
print("Train AUC score: ", train_auc_score)

# Evaluate the model on the test set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test_processed)[:, 1]
test_auc_score = roc_auc_score(y_test, y_pred_proba)
print("Test AUC score: ", test_auc_score)

# Evaluate the model on the oot set
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("TRAIN GINI score: ", round(2*train_auc_score-1,3))
print("Test GINI score: ", round(2*test_auc_score-1,3))
print("OOT GINI score: ", round(2*oot_auc_score-1,3))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best parameters found:  {'subsample': 0.6, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6}
Best AUC score:  0.5178713409965182
Train AUC score:  0.7490754486408927
Test AUC score:  0.6186058618355978
OOT AUC score:  0.615086489825862
TRAIN GINI score:  0.498
Test GINI score:  0.237
OOT GINI score:  0.23


## prepare model artefact to save

In [13]:
model_artefact = {}

model_artefact['model'] = best_model
model_artefact['model_version'] = "credit_model_"+config["model_train_date_str"].replace('-','_')
model_artefact['preprocessing_transformers'] = {}
model_artefact['preprocessing_transformers']['stdscaler'] = transformer_stdscaler
model_artefact['data_dates'] = config
model_artefact['data_stats'] = {}
model_artefact['data_stats']['X_train'] = X_train.shape[0]
model_artefact['data_stats']['X_test'] = X_test.shape[0]
model_artefact['data_stats']['X_oot'] = X_oot.shape[0]
model_artefact['data_stats']['y_train'] = round(y_train.mean(),2)
model_artefact['data_stats']['y_test'] = round(y_test.mean(),2)
model_artefact['data_stats']['y_oot'] = round(y_oot.mean(),2)
model_artefact['results'] = {}
model_artefact['results']['auc_train'] = train_auc_score
model_artefact['results']['auc_test'] = test_auc_score
model_artefact['results']['auc_oot'] = oot_auc_score
model_artefact['results']['gini_train'] = round(2*train_auc_score-1,3)
model_artefact['results']['gini_test'] = round(2*test_auc_score-1,3)
model_artefact['results']['gini_oot'] = round(2*oot_auc_score-1,3)
model_artefact['hp_params'] = random_search.best_params_


pprint.pprint(model_artefact)

{'data_dates': {'model_train_date': datetime.datetime(2024, 9, 1, 0, 0),
                'model_train_date_str': '2024-09-01',
                'oot_end_date': datetime.datetime(2024, 8, 31, 0, 0),
                'oot_period_months': 2,
                'oot_start_date': datetime.datetime(2024, 7, 1, 0, 0),
                'train_test_end_date': datetime.datetime(2024, 6, 30, 0, 0),
                'train_test_period_months': 12,
                'train_test_ratio': 0.8,
                'train_test_start_date': datetime.datetime(2023, 7, 1, 0, 0)},
 'data_stats': {'X_oot': 1003,
                'X_test': 1192,
                'X_train': 4766,
                'y_oot': np.float64(0.29),
                'y_test': np.float64(0.28),
                'y_train': np.float64(0.28)},
 'hp_params': {'colsample_bytree': 0.6,
               'gamma': 0.1,
               'learning_rate': 0.1,
               'max_depth': 3,
               'min_child_weight': 5,
               'n_estimators': 50,
        

## save artefact to model bank

In [14]:
# create model_bank dir
model_bank_directory = "model_bank/"

if not os.path.exists(model_bank_directory):
    os.makedirs(model_bank_directory)

In [15]:
# Full path to the file
file_path = os.path.join(model_bank_directory, model_artefact['model_version'] + '.pkl')

# Write the model to a pickle file
with open(file_path, 'wb') as file:
    pickle.dump(model_artefact, file)

print(f"Model saved to {file_path}")


Model saved to model_bank/credit_model_2024_09_01.pkl


## test load pickle and make model inference

In [16]:
# Load the model from the pickle file
with open(file_path, 'rb') as file:
    loaded_model_artefact = pickle.load(file)

y_pred_proba = loaded_model_artefact['model'].predict_proba(X_oot_processed)[:, 1]
oot_auc_score = roc_auc_score(y_oot, y_pred_proba)
print("OOT AUC score: ", oot_auc_score)

print("Model loaded successfully!")

OOT AUC score:  0.615086489825862
Model loaded successfully!
